In [ ]:
# auto format code at cell execution
%load_ext lab_black

root_location = "../../"
import sys

sys.path.append(root_location)

import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import matplotlib as mpl
import numpy as np

from src.tools.pre_processing import read_csv_as_geopandas, transform_to_geo_data
from src.tools.visualising import show_data_over_roads

%matplotlib inline

# Parameters

In [ ]:
input_file_name = root_location + "data/samples/year_2018_country_Belgium.csv"

# Load data

In [ ]:
%%time

gdf = read_csv_as_geopandas(input_file_name)

# Visualisation

## Heatmap to find dense areas

In [ ]:
%%time

count, x, y, _ = plt.hist2d(
        x=gdf.longitudeE7,
        y=gdf.latitudeE7,
        bins=3000,
        norm=mpl.colors.LogNorm()
    )

In [ ]:
LIMIT = 300
sum(sum(count > LIMIT))

In [ ]:
def select_and_render(ix, iy, ax=None, gdf=gdf):
    selection = gdf[
        (gdf.longitudeE7 > x[ix])
        & (gdf.longitudeE7 < x[ix + 1])
        & (gdf.latitudeE7 > y[iy])
        & (gdf.latitudeE7 < y[iy + 1])
    ]
    show_data_over_roads(selection, ax=ax, repository_root_location=root_location)

In [ ]:
locations = tuple(zip(*np.where(count > LIMIT)))

## 1 Example (example 0 is in the middle of nowhere)

In [ ]:
%%time
select_and_render(*locations[1])

## Rest of the examples
(sqrt_num_figs = 2) demonstrates what this does in 27s  
(sqrt_num_figs = 6) shows all (30) cases in 10min

In [ ]:
%%time

sqrt_num_figs = 2

fig, axes = plt.subplots(sqrt_num_figs, sqrt_num_figs, figsize=(5 * sqrt_num_figs, 4 * sqrt_num_figs))

for (fig_num, (ix, iy)) in enumerate(locations[:sqrt_num_figs**2]):
    print(f"start rendering figure {fig_num}")
    %time select_and_render(ix, iy, axes[fig_num // sqrt_num_figs][fig_num % sqrt_num_figs])
    print(f"done rendering figure {fig_num}")

## Dense locations on Belgian map

In [ ]:
fig, ax = plt.subplots()

show_data_over_roads(
    transform_to_geo_data(
        pd.DataFrame(
            {
                "longitudeE7": x[[loc[0] for loc in locations]],
                "latitudeE7": y[[loc[1] for loc in locations]],
            }
        )
    ),
    ax=ax,
    repository_root_location=root_location,
)

## Coordinates of locations for checking on google maps

In [ ]:
tuple(
    zip(
        y[[loc[1] for loc in locations]] / 10_000_000,
        x[[loc[0] for loc in locations]] / 10_000_000,
    )
)